# Libraries and Data (TURN ON GPU)

In [1]:
# Mount Google Drive to access files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Path to the folder
%cd /content/drive/MyDrive/Python - Time Series Forecasting/Deep Learning for Time Series Forecasting /Multiple Series-LSTM

/content/drive/MyDrive/Python - Time Series Forecasting/Deep Learning for Time Series Forecasting /Multiple Series-LSTM


In [3]:
# Install the darts library
!pip install darts -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.6/200.6 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.4/825.4 kB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.0/340.0 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 100.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [73]:
# Standard Libraries
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import month_plot, quarter_plot
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, mean_absolute_percentage_error
import numpy as np
from sklearn.model_selection import ParameterGrid

# Darts functions
from darts.timeseries import TimeSeries
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.dataprocessing.transformers import Scaler
from darts.models import RNNModel

In [76]:
# Load the CSV data into a NumPy array
df = pd.read_csv("Hourly-train.csv", header = None).values.squeeze()
df

array([['V1', 'V2', 'V3', ..., 'V959', 'V960', 'V961'],
       ['H1', '605', '586', ..., nan, nan, nan],
       ['H2', '3124', '2990', ..., nan, nan, nan],
       ...,
       ['H412', '19', '30', ..., '73', '39', '36'],
       ['H413', '21', '16', ..., '53', '38', '46'],
       ['H414', '15', '13', ..., '35', '26', '17']], dtype=object)

In [77]:
df.shape

(415, 961)

In [78]:
# Excluding the first row and first column
df = df[1:, 1:].astype(np.float64)
df

array([[ 605.,  586.,  586., ...,   nan,   nan,   nan],
       [3124., 2990., 2862., ...,   nan,   nan,   nan],
       [1828., 1806., 1897., ...,   nan,   nan,   nan],
       ...,
       [  19.,   30.,   12., ...,   73.,   39.,   36.],
       [  21.,   16.,   17., ...,   53.,   38.,   46.],
       [  15.,   13.,   13., ...,   35.,   26.,   17.]])

In [79]:
df.shape

(414, 960)

In [80]:
# Transpose the data
df = df.T
df

array([[ 605., 3124., 1828., ...,   19.,   21.,   15.],
       [ 586., 2990., 1806., ...,   30.,   16.,   13.],
       [ 586., 2862., 1897., ...,   12.,   17.,   13.],
       ...,
       [  nan,   nan,   nan, ...,   73.,   53.,   35.],
       [  nan,   nan,   nan, ...,   39.,   38.,   26.],
       [  nan,   nan,   nan, ...,   36.,   46.,   17.]])

In [81]:
df.shape

(960, 414)

In [82]:
# Check which wos have nan values
na_rows = np.isnan(df).any(axis = 1)
np.where(na_rows)

(array([700, 701, 702, 703, 704, 705, 706, 707, 708, 709, 710, 711, 712,
        713, 714, 715, 716, 717, 718, 719, 720, 721, 722, 723, 724, 725,
        726, 727, 728, 729, 730, 731, 732, 733, 734, 735, 736, 737, 738,
        739, 740, 741, 742, 743, 744, 745, 746, 747, 748, 749, 750, 751,
        752, 753, 754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 764,
        765, 766, 767, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777,
        778, 779, 780, 781, 782, 783, 784, 785, 786, 787, 788, 789, 790,
        791, 792, 793, 794, 795, 796, 797, 798, 799, 800, 801, 802, 803,
        804, 805, 806, 807, 808, 809, 810, 811, 812, 813, 814, 815, 816,
        817, 818, 819, 820, 821, 822, 823, 824, 825, 826, 827, 828, 829,
        830, 831, 832, 833, 834, 835, 836, 837, 838, 839, 840, 841, 842,
        843, 844, 845, 846, 847, 848, 849, 850, 851, 852, 853, 854, 855,
        856, 857, 858, 859, 860, 861, 862, 863, 864, 865, 866, 867, 868,
        869, 870, 871, 872, 873, 874, 875, 876, 877

In [83]:
# Remove the columns with NaN value
df = df[~na_rows]
df

array([[ 605., 3124., 1828., ...,   19.,   21.,   15.],
       [ 586., 2990., 1806., ...,   30.,   16.,   13.],
       [ 586., 2862., 1897., ...,   12.,   17.,   13.],
       ...,
       [ 752., 3460., 1581., ...,   18.,   17.,   13.],
       [ 739., 3248., 1570., ...,   13.,   16.,   17.],
       [ 684., 3105., 1518., ...,   17.,   15.,   21.]])

In [84]:
# Display how many time series
len(df)

700

In [85]:
# Preview df shape
df.shape

(700, 414)